In [20]:
import requests
import json
import pandas

In [21]:
df = pandas.read_csv('/Users/wikus/code/wikusbrink.com/PG.csv')
df.columns = [c.replace(' ', '_') for c in df.columns]
df[['Afternoon_exercise_points', 'Morning_exercise_points', 'Alcohol_units']] = df[['Afternoon_exercise_points', 'Morning_exercise_points', 'Alcohol_units']].fillna(value=0)
df = df.fillna('undefined')


In [22]:
dicts = df.to_dict(orient='records')
dicts[0]

{'Day': 'Wednesday',
 'Date': '2018-09-05',
 'Morning_exercise_type': 'Light strength training',
 'Morning_exercise_amount': '5kg, 10 pu, 5kg',
 'Morning_exercise_points': 1.0,
 'Afternoon_exercise_type': 'undefined',
 'Afternoon_exercise_amount': 'undefined',
 'Afternoon_exercise_points': 0.0,
 'Exercise_points': 1.0,
 'Exercise_rolling_average': 1.0,
 'Feeding_window_start': '12:20',
 'Feeding_window_end': '19:20',
 'Feeding_window_length': '7:00:00',
 'Fasting_window': 'undefined',
 'Clean_eating': False,
 'Alcohol_units': 0.0,
 'Clean_day': 0.0,
 'Clean_streak': 0.0,
 'Morning_weight': 'undefined',
 'Weight_rolling_average': 'undefined',
 '10%_activity': 'undefined',
 '10%_activity_done': False,
 'Notes': 'undefined',
 '16:00:00': '16:00:00',
 '8:00:00': '8:00:00'}

In [25]:
url = "http://ec2-13-211-175-189.ap-southeast-2.compute.amazonaws.com:5001/api/lifestyle/"
#url = "http://localhost:5000/api/lifestyle/"

headers = {
    'content-type': "application/json",
    'cache-control': "no-cache",
}

for i in dicts:
    print(i['Date'])
    if i['Date'] == '2018-10-22':
        break
        
    weight = i['Morning_weight']
    if weight != 'undefined':
        weight = weight
    else:
        weight = None
        
    morning_type = i['Morning_exercise_type']
    if morning_type == 'undefined':
        morning_type = None
    else:
        morning_type = i['Morning_exercise_type'].replace('Run', 'run').replace('Hike', 'hike').replace('Paddle', 'paddle').replace('Light strength training', 'strength').replace('Pilates', 'pilates')
     
    afternoon_type = i['Afternoon_exercise_type']
    if afternoon_type == 'undefined':
        afternoon_type = None
    else:
        afternoon_type = i['Afternoon_exercise_type'].replace('Run', 'run').replace('Hike', 'hike').replace('Paddle', 'paddle').replace('Light strength training', 'strength').replace('Pilates', 'pilates')
     
    alcohol = i['Alcohol_units']
    if alcohol != 'undefined':
        alcohol = alcohol
    else:
        alcohol = None
    
    fws = i['Feeding_window_start']
    if fws != 'undefined':
        fws = fws
    else:
        fws = None
    
    fwe = i['Feeding_window_end']
    if fwe != 'undefined':
        fwe = fwe
    else:
        fwe = None
        
        
    notes = i['Notes']
    if notes != 'undefined':
        notes = notes
    else:
        notes = None
        
    mep = i['Morning_exercise_points']
    if mep != 'undefined':
        mep = mep
    else:
        mep = None
        
    mea = i['Morning_exercise_amount']
    if mea != 'undefined':
        mea = mea
    else:
        mea = None
        
    aep = i['Afternoon_exercise_points']
    if aep != 'undefined':
        aep = aep
    else:
        aep = None
        
    aea = i['Afternoon_exercise_amount']
    if aea != 'undefined':
        aea = aea
    else:
        aea = None
    
    d = {
        'date': i['Date'],
        'data': {
            'weight': weight,
            'alcohol': int(alcohol),
            'feedingWindowStart': fws,
            'feedingWindowEnd': fwe,
            'cleanEating': i['Clean_eating'], #.replace('False', False).replace('True', True),
            'notes': notes,
            'morningExercise': {
                'type': morning_type,
                'points': int(mep),
                'notes': mea
            },
            'afternoonExercise': {
                'type': afternoon_type,
                'points': int(aep),
                'notes': aea
            }
        }
    }
    payload = json.dumps(d)
    response = requests.request("PUT", url+i['Date'], data=payload, headers=headers)
    
    

2018-09-05
2018-09-06
2018-09-07
2018-09-08
2018-09-09
2018-09-10
2018-09-11
2018-09-12
2018-09-13
2018-09-14
2018-09-15
2018-09-16
2018-09-17
2018-09-18
2018-09-19
2018-09-20
2018-09-21
2018-09-22
2018-09-23
2018-09-24
2018-09-25
2018-09-26
2018-09-27
2018-09-28
2018-09-29
2018-09-30
2018-10-01
2018-10-02
2018-10-03
2018-10-04
2018-10-05
2018-10-06
2018-10-07
2018-10-08
2018-10-09
2018-10-10
2018-10-11
2018-10-12
2018-10-13
2018-10-14
2018-10-15
2018-10-16
2018-10-17
2018-10-18
2018-10-19
2018-10-20
2018-10-21
2018-10-22


In [28]:
data = requests.request("GET", url, data=payload, headers=headers).json()

In [32]:
import json
import datetime
with open('../backups/lifestyle_'+datetime.datetime.now().isoformat()[:10] + '.json', 'w') as fp:
    json.dump(data, fp)